<a href="https://colab.research.google.com/github/SaulHuitzil/Boolean-Networks/blob/main/Boolean_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from typing import List, Dict
import random

In [65]:
#!pip install import-ipynb

In [66]:
#!wget -O Node.ipynb https://raw.githubusercontent.com/SaulHuitzil/Boolean-Networks/refs/heads/main/Node.ipynb

In [81]:
# import import_ipynb
from Node import Node

In [71]:
from typing import List
import random
import math


class Network:
    """
    Represents a network of nodes (`Node`), designed for modeling Boolean systems.
    """

    def __init__(self):
        # List of nodes in the network.
        self.Nodes: List[Node] = []

        # Probabilities for mutations and activations.
        self.p_add_connection = 0.5
        self.p_activate_node = 0.5
        # Random number generator for consistent randomness.
        self.random = random.Random()

    def generate_random_state(self):
        """
        Generates a random binary string representing the initial state of the network.
        """
        return ''.join(random.choice('01') for _ in range(len(self.Nodes)))

    def clone(self) -> 'Network':
        """
        Creates a deep copy of the current network, including all nodes.
        """
        cloned_network = Network()
        cloned_network.Nodes = [node.clone() for node in self.Nodes]
        return cloned_network

    def create_network(self, num_nodes: int, connectivity: float):
        """
        Initializes the network with a specified number of nodes and connectivity.
        """
        self.Nodes = []
        for _ in range(num_nodes):
            node = Node()
            node.create_entries(connectivity, num_nodes)
            #node.create_entries_with_poisson(num_nodes, connectivity)
            node.create_boolean_function()
            self.Nodes.append(node)

    def update_state(self, initial_state: str) -> str:
        """
        Updates the state of the network based on the current Boolean functions of the nodes.
        :param initial_state: Binary string representing the current state of all nodes.
        :return: Binary string representing the updated state of all nodes.
        """
        updated_state = []
        for node in self.Nodes:
            # Gather the state of the node's inputs.
            configuration = [initial_state[entry] for entry in node.Entries]
            configuration_str = ''.join(configuration)
            # Determine the output of the node based on its Boolean function.
            updated_state.append(node.BF[configuration_str])
        return ''.join(updated_state)

    def average_connectivity(self) -> float:
        """
        Calculates the average connectivity of the network.
        """
        return float(sum(len(node.Entries) for node in self.Nodes)) / len(self.Nodes)

    def fraction_of_ones(self) -> float:
        """
        Calculates the fraction of '1's in the Boolean functions of all nodes.
        """
        all_outputs = ''.join(''.join(node.BF.values()) for node in self.Nodes)
        return all_outputs.count('1') / len(all_outputs) if all_outputs else 0.0

    def sensitivity(self) -> float:
        """
        Estimates the sensitivity of the network based on its connectivity and Boolean function outputs.
        """
        k = self.average_connectivity()
        p = self.fraction_of_ones()
        return 2 * p * (1 - p) * k

    def mutate_conecction(self):
        """
        Applies mutations to the network by rewiring node connections.
        """
        node = random.choice(self.Nodes)
        if self.random.random() < 0.5:
            # Add a new connection.
            available_nodes = [i for i in range(len(self.Nodes)) if i not in node.Entries]
            if available_nodes:
                new_entry = self.random.choice(available_nodes)
                node.add_entry(new_entry)
        else:
            # Remove an existing connection.
            if len(node.Entries) > 1:
                entry_to_remove = self.random.choice(node.Entries)
                node.remove_entry(entry_to_remove)


    def mutate_functions(self):
        """
        Applies mutations to the network by fliping node Boolean functions values.
        """
        node = random.choice(self.Nodes)
        key = random.choice(list(node.BF.keys()))
        node.BF[key] = '1' if node.BF[key] == '0' else '0'


    def mutate(self):
      """
      Applies a random mutation to the network.
      """
      if self.random.random() < 0.5:
        self.mutate_functions()
      else:
        self.mutate_conecction()



In [80]:
if __name__ == "__main__":
    import random

    # Seed for reproducibility
    #random.seed(42)

    # Create a network
    num_nodes = 10
    connectivity = 2

    print("=== Testing Network Creation ===")
    network = Network()
    network.create_network(num_nodes, connectivity)
    print(f"Created network with {len(network.Nodes)} nodes.")
    print(f"Average connectivity: {network.average_connectivity():.2f}")
    print(f"Fraction of ones: {network.fraction_of_ones():.2f}")
    print(f"Sensitivity: {network.sensitivity():.2f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")

    # Test state update
    print("\n=== Testing State Update ===")
    state = network.generate_random_state()
    for t in range(10):
      print(f"state_{t}: {state}")
      state = network.update_state(state)

    # Clone the network
    print("\n=== Testing Network Cloning ===")
    cloned_network = network.clone()
    print("Cloned network created.")
    print(f"Cloned average connectivity: {cloned_network.average_connectivity():.5f}")
    print(f"Fraction of ones: {cloned_network.fraction_of_ones():.5f}")
    print(f"Sensitivity: {cloned_network.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(cloned_network.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")

    # Mutate the network - Connection
    print("\n=== Testing Network Mutation - Connection ===")
    network_mutated_connection = network.clone()
    network_mutated_connection.mutate_conecction()
    print(f"Post-mutation average connectivity: {network_mutated_connection.average_connectivity():.5f}")
    print(f"Fraction of ones: {network_mutated_connection.fraction_of_ones():.5f}")
    print(f"Sensitivity: {network_mutated_connection.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network_mutated_connection.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")


    # Mutate the network - Boolean Function
    print("\n=== Testing Network Mutation - Boolean Function ===")
    network_mutated_function = network.clone()
    network_mutated_function.mutate_functions()
    print(f"Post-mutation average connectivity: {network_mutated_function.average_connectivity():.5f}")
    print(f"Fraction of ones: {network_mutated_function.fraction_of_ones():.5f}")
    print(f"Sensitivity: {network_mutated_function.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network_mutated_function.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")


=== Testing Network Creation ===
Created network with 10 nodes.
Average connectivity: 2.00
Fraction of ones: 0.55
Sensitivity: 0.99
Nodes and their connections:
  Node 0: Entries=[8, 0], BF={'00': '0', '01': '0', '10': '1', '11': '1'}
  Node 1: Entries=[0, 1], BF={'00': '1', '01': '0', '10': '0', '11': '1'}
  Node 2: Entries=[0, 3], BF={'00': '0', '01': '1', '10': '0', '11': '1'}
  Node 3: Entries=[2, 3], BF={'00': '1', '01': '1', '10': '1', '11': '0'}
  Node 4: Entries=[1, 8], BF={'00': '1', '01': '0', '10': '0', '11': '1'}
  Node 5: Entries=[3, 1], BF={'00': '1', '01': '1', '10': '1', '11': '0'}
  Node 6: Entries=[4, 7], BF={'00': '1', '01': '0', '10': '1', '11': '1'}
  Node 7: Entries=[6, 2], BF={'00': '1', '01': '0', '10': '1', '11': '0'}
  Node 8: Entries=[1, 9], BF={'00': '1', '01': '0', '10': '1', '11': '0'}
  Node 9: Entries=[6, 5], BF={'00': '1', '01': '0', '10': '0', '11': '0'}

=== Testing State Update ===
state_0: 1111000011
state_1: 1110101001
state_2: 0101011000
state_3: 